In [2]:
import time as tm
import numpy as np
import pandas as pd
import sys

In [2]:
pd.set_option('display.width', 150)
pd.set_option('display.max_colwidth', 120)
pd.set_option('display.max_columns', 90)

In [ ]:
df = pd.read_csv("dirty/primary_debate01.csv")

In [ ]:
del df['SentenceNo']
df['Text'].replace(to_replace="\(.*\)", value="", inplace=True, regex=True)
df['Text'].replace(to_replace="", value=np.nan, inplace=True, regex=True)
df = df.dropna()
print df.shape

In [ ]:
cdf = df[df['Text'].map(len) > 10]
cdf = cdf.groupby(['SequenceNo', 'Speaker'])['Text'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
cdf.rename(columns={"Text": "answer"}, inplace=True)
cdf['query'] = np.where(cdf['answer'].str.contains("Mr. Trump, your"), cdf['answer'].shift(2), cdf['answer'].shift(1))
print cdf.shape

In [ ]:
cdf = cdf[cdf['Speaker'] == "TRUMP"]
cdf['query'].replace(to_replace="Mr\. Trump\, ", value="", inplace=True, regex=True)
print cdf.shape

In [ ]:
files = ["01", "02", "03", "04", "05", "06", "07" "08", "09", "10", "11", "12"]
# file 7 is actually the presidential debates
# files = ["editorial", "matthews", "military"]
all_dfs = []

for file in files:
    filename = "debates/primary_debate"+file+".csv"
    df = pd.read_csv(filename)
    df.rename(columns={"2":"speaker", "3":"answer"}, inplace=True)
    df['query'] = np.where(df['answer'].str.contains("Mr. Trump, your"), df['answer'].shift(2), df['answer'].shift(1))
    df = df[df['speaker'] == "TRUMP"]
    df = df[["answer", "query", "source"]]
    all_dfs.append(df)
    print df.shape

final_df = pd.concat(all_dfs)
print final_df.shape
# final_df.to_csv("discussions.csv", index=False, header=True)

In [ ]:
op = pd.read_csv("clean/discussions.csv")
op

In [22]:
files = ["07", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
# file 7 is actually the presidential debates
# files = ["editorial", "matthews", "military"]
all_dfs = []

for file in files:
    filename = "debates/primary_debate"+file+".csv"
    df = pd.read_csv(filename)

    df['Text'].replace(to_replace="\(.*\)", value="", inplace=True, regex=True)
    df['Text'].replace(to_replace="", value=np.nan, inplace=True, regex=True)
    df = df.dropna()
    df = df[df['Text'].map(len) > 10]
    df.rename(columns={"Text": "answer"}, inplace=True)
    
    df['query'] = np.where(df['answer'].str.contains("Mr. Trump, your"), df['answer'].shift(2), df['answer'].shift(1))

    if file == "07":
        df = df[df['Speaker'] == "Trump"]
    else:
        df = df[df['Speaker'] == "TRUMP"]
    df['query'].replace(to_replace="Mr\. Trump\, ", value="", inplace=True, regex=True)
    del df['Speaker']
    if file == "07":
        del df['Line']
        del df['source']
    else:
        del df['SentenceNo']
        del df['SequenceNo']
    print df.shape
    all_dfs.append(df)

final_df = pd.concat(all_dfs)
print final_df.shape

(334, 2)
(168, 2)
(50, 2)
(53, 2)
(76, 2)
(94, 2)
(334, 2)
(76, 2)
(110, 2)
(239, 2)
(229, 2)
(101, 2)
(1864, 2)


In [23]:
final_df.head()

,answer,query
10,"Thank you, Lester. Our jobs are fleeing the co...","Secretary Clinton, thank you. the same questio..."
11,"We cannot let it happen. Under my plan, I'll b...","Thank you, Lester. Our jobs are fleeing the co..."
15,"Well, for one thing -- and before we start on ...",Let me follow up with if you can. You've talke...
17,"Secretary Clinton and others, politicians, sho...","Let me interrupt just a moment, but..."
19,"Well, the first thing you do is don't let the ...","Back to the question, though. How do you bring..."


In [ ]:
def recursiveSplit(line, sentences):
    if (len(line) <= 10):
        return []
    elif (len(sentences) < 2) or (len(line) < 250):
        return [line]
    else:
        allLengths = [len(s) for s in sentences]
        leftSide = 0
        rightSide = sum(allLengths)
        diffs = []
        for l in allLengths:
            leftSide = leftSide + l
            rightSide = rightSide - l
            diffs.append(abs(leftSide - rightSide))
        splitIndex = diffs.index(min(diffs))

        previous = ' '.join([str(s) for s in sentences[:splitIndex+1]])
        previousSent = sentences[:splitIndex+1]
        upcoming = ' '.join([str(s) for s in sentences[splitIndex+1:]])
        upcomingSent = sentences[splitIndex+1:]

        splitSentence = []
        splitSentence += recursiveSplit(previous, previousSent)
        splitSentence += recursiveSplit(upcoming, upcomingSent)   
        return splitSentence

In [ ]:
allText = []
for filename in files:
    toRead = "speeches/"+filename+".txt"
    fileText = []
    with open (toRead, "r") as myfile:
        for line in myfile:
            text = nlp(unicode(line))
            sentences = [s for s in text.sents]
            fileText += recursiveSplit(line, sentences)
    print filename, ": ", len(fileText)

    allText += fileText
print "FINAL LENGTH: ", len(allText)

with open('speeches.csv', 'wb+') as f:
    myWriter = csv.writer(f)
    for q, a in zip(allText, allText[1:]):
        myWriter.writerow([a, q, 'speeches'])

In [24]:
final_df.to_csv("scrub.csv", index=False, header=True)